In [ ]:
## Detect turn and extract turn angle 
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
from matplotlib.transforms import Bbox
import pickle

import seaborn as sns; sns.set(color_codes=True)
from scipy.stats import gaussian_kde
from scipy import signal
import pandas as pd
from scipy.signal import find_peaks
import scipy as sp
import scipy.stats as stats

datafolder = "/Users/your/datafolder"
savefolder = "/Users/your/savefolder"

## name of each fish data
fishes_setup = [

"2025_05_23_fish001",
"2025_05_23_fish002",
"2025_05_23_fish003",
"2025_05_23_fish004",
"2025_05_23_fish005",
"2025_05_23_fish006",
"2025_05_23_fish007",
"2025_05_23_fish008",
"2025_05_23_fish009",
"2025_05_23_fish010",
"2025_05_23_fish011",
"2025_05_23_fish012",
"2025_05_23_fish013",
"2025_05_23_fish014",
"2025_05_23_fish015",
"2025_05_23_fish016",
"2025_05_23_fish017",
"2025_05_23_fish018",
"2025_05_23_fish019",
"2025_05_23_fish020",
"2025_05_23_fish021",

]

## saving name of the data
exp_name = "df_medaka_black_stimRest_sine_pulse_correct_alternate"

patterns = [0,1,2,3,4,5,6]

directions = [0,1]

stimulus_pattern = []

for pattern in patterns:
    for direction in directions:
        stimulus_pattern.append([pattern, direction])


print(stimulus_pattern)
columns_name = ["fish_index", "trial_index", "orderIndex","stimulus_index", "start_time_index", "center_distance","start_position_x", "start_position_y", "head_angle",
                "bout_angle", "accumulated_angle", "fishName"]

data_frame = pd.DataFrame(columns=columns_name)

stim_data_stimIndex = []
stim_data_fishIndex = []
stim_data_trialIndex = []
stim_data_start_time = []
stim_data_center = []
stim_data_start_position_x = []
stim_data_start_position_y = []
stim_data_angle_difference = []
stim_data_headAngle =[]
stim_data_accumAngle = []

trial_num = 20
start_timing = 0
stop_timing = 25
dt = 1/90


for setup_ID in [0]:

    if setup_ID == 0:
        fishes = fishes_setup

    for stimulus_n in range(len(stimulus_pattern)):
        pattern, direction = stimulus_pattern[stimulus_n]
        print(pattern, direction, stimulus_n)

        for i, fish in enumerate(fishes):

            for trial in range(0, trial_num):
                print('fish:', fish, "trial", trial, "stimulus number:", stimulus_n)

                stimulus = stimulus_n

                count_turn = []
                accu_ori_changes = []
               
                file = r"%s/%s/raw_data/trial%03d.dat" % (datafolder, fish, trial)

                try:
                    f = open(file, 'rb')
                    data = pickle.load(f)
                    f.close()
                except:
                    break


                ########3 for orderIndex
                frame_num = np.zeros(len(stimulus_pattern))
                for p in range(len(stimulus_pattern)):
                    frame_num[p] = data[f'raw_stimulus_{p:03d}']['camera_framenum'][0]
                order = np.argsort(frame_num)
                orderindex = order[stimulus_n]
                #########


                t_start = start_timing
                t_stop = stop_timing

                start_times = data["raw_stimulus_{:03d}".format(stimulus)][
                    "timestamp"]  # "bouts_start_stimulus_{:03d}".format(stim)
                # fish_movie_framenum
                fish_movie_framenum = data["raw_stimulus_{:03d}".format(stimulus)][
                    "fish_movie_framenum"]
                ind1 = np.where((np.array(start_times) > (t_start + dt / 2)) & (
                        np.array(start_times) < (t_stop - dt / 2)))
                index_ind1 = np.array(fish_movie_framenum[ind1])

                start_times = data["raw_stimulus_{:03d}".format(stimulus)][
                    "timestamp"]  # "bouts_start_stimulus_{:03d}".format(stim)

                start_times = start_times[ind1]

                x = data["raw_stimulus_{:03d}".format(stimulus)]["fish_position_x"][ind1]
                y = data["raw_stimulus_{:03d}".format(stimulus)]["fish_position_y"][ind1]
                fish_accumulated_orientation = data["raw_stimulus_{:03d}".format(stimulus)][
                    "fish_accumulated_orientation"][ind1]

                fish_headAngle = data["raw_stimulus_{:03d}".format(stimulus)]["fish_orientation"][ind1]

                ind1_list = np.array(ind1[0]).tolist()

                dis_center = 60 * np.sqrt(x ** 2 + y ** 2)

                diff_tail_deflection_trial = []
                diff_fish_accumulated_orientation = []
                diff_x = []
                diff_y = []
                diff_dis_center = []
                diff_start_times = []
                diff = 20
                speed = []

                for ind_n in range(len(ind1_list)-diff):
                    # diff_tail_deflection_trial.append(tail_deflection_trial[ind_n + diff] - tail_deflection_trial[ind_n])
                    diff_fish_accumulated_orientation.append(fish_accumulated_orientation[ind_n + diff] - fish_accumulated_orientation[ind_n])
                    diff_x.append(x[ind_n + diff] - x[ind_n])
                    diff_y.append(y[ind_n + diff] - y[ind_n])
                    diff_dis_center.append(dis_center[ind_n + diff] - dis_center[ind_n])
                    diff_start_times.append(start_times[ind_n])
                    distance_move = ((x[ind_n + diff] - x[ind_n])**2 + (y[ind_n + diff] - y[ind_n])**2)**(1/2)/(diff/90)
                    speed.append(distance_move)


                peaks, _ = find_peaks(speed, distance=50, height=0.05)
                speed_peaks = np.array(speed)[peaks.astype(int)]

                good_speed_peaks = []
                time_peaks = []
                peak_turns = []


                turn_start_x = []
                turn_start_y = []
                turn_dis_center = []
                turn_accumulatedAngle = []
                turn_counter = 0
                turn_head_orientation = []

                for p in range(len(speed_peaks)):

                    if speed_peaks[p] < 1.5:
                        good_speed_peaks.append(speed_peaks[p])
                        time_peaks.append(diff_start_times[peaks[p]])#append(np.array(diff_start_times)[peaks.astype(int)[p]])
                        # peak_turns.append(diff_fish_accumulated_orientation[peaks[p]])#(np.array(diff_fish_accumulated_orientation)[peaks.astype(int)[p]])
                        peak_turns.append(fish_accumulated_orientation[peaks[p]+int(diff/2)] - fish_accumulated_orientation[peaks[p]-int(diff/2)])
                        turn_start_x.append(x[peaks[p]])
                        turn_start_y.append(y[peaks[p]])
                        turn_dis_center.append(dis_center[peaks[p]])
                        turn_accumulatedAngle.append(fish_accumulated_orientation[peaks[p]])
                        turn_counter += 1
                        turn_head_orientation.append(fish_headAngle[peaks[p]])

                    else:
                        pass

                turn_angles = peak_turns
                turn_start_time = time_peaks

                stimIndex = [stimulus_n] * turn_counter
                fishName = [fish] * turn_counter
                fishIndex = [i] * turn_counter
                trialIndex = [trial] * turn_counter

                orderIndex = [orderindex] * turn_counter


                d = {"orderIndex": orderIndex,
                     "stimulus_index": stimIndex, "fish_index": fishIndex, "trial_index": trialIndex,
                     "head_angle": turn_head_orientation, "accumulated_angle": turn_accumulatedAngle,
                     "start_time_index": turn_start_time, "bout_angle": turn_angles, "center_distance": turn_dis_center,
                     "fishName": fishName,
                     }


                small_data_frame = pd.DataFrame(d, columns=columns_name)
                data_frame = data_frame.append(small_data_frame, ignore_index=True)


    print(data_frame.shape)
    data_frame.to_pickle("%s/df_%s.pkl" % (savefolder, exp_name))
    data_frame.to_csv("%s/df_%s.csv" % (savefolder, exp_name))




